In [23]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
df = pd.read_csv('G:/New 20121 Dekstop Files/thesis 2021/employee attrition dataset/employee_attrition.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [3]:
df['Attrition'].replace({'Yes':1,'No':0}, inplace = True)

In [4]:
df.Attrition

0       1
1       0
2       1
3       0
4       0
       ..
1465    0
1466    0
1467    0
1468    0
1469    0
Name: Attrition, Length: 1470, dtype: int64

In [5]:
label_encoder =  preprocessing.LabelEncoder()
df['BusinessTravel'] = label_encoder.fit_transform(df['BusinessTravel'])
df['Department'] = label_encoder.fit_transform(df['Department'])
df['EducationField'] = label_encoder.fit_transform(df['EducationField'])
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['JobRole'] = label_encoder.fit_transform(df['JobRole'])
df['MaritalStatus'] = label_encoder.fit_transform(df['MaritalStatus'])
df['Over18'] = label_encoder.fit_transform(df['Over18'])
df['OverTime'] = label_encoder.fit_transform(df['OverTime'])

In [6]:
df.sample(7)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
249,45,0,1,1199,1,7,4,1,1,341,...,4,80,1,9,1,3,3,2,0,2
208,40,0,0,1151,1,9,5,1,1,287,...,4,80,1,5,5,1,3,2,0,2
675,31,0,2,154,2,7,4,1,1,941,...,2,80,0,13,2,4,7,7,5,2
1310,58,0,1,1216,1,15,4,1,1,1837,...,2,80,0,23,3,3,2,2,2,2
966,58,1,2,601,1,7,4,3,1,1360,...,4,80,0,31,0,2,10,9,5,9
1027,34,0,2,401,1,1,3,1,1,1447,...,1,80,1,7,2,2,5,4,0,2
1331,48,0,2,1224,1,10,3,1,1,1867,...,4,80,0,29,3,3,22,10,12,9


In [7]:
for col in df:
    print(f'{col}: {df[col].unique()}') 

Age: [41 49 37 33 27 32 59 30 38 36 35 29 31 34 28 22 53 24 21 42 44 46 39 43
 50 26 48 55 45 56 23 51 40 54 58 20 25 19 57 52 47 18 60]
Attrition: [1 0]
BusinessTravel: [2 1 0]
DailyRate: [1102  279 1373 1392  591 1005 1324 1358  216 1299  809  153  670 1346
  103 1389  334 1123 1219  371  673 1218  419  391  699 1282 1125  691
  477  705  924 1459  125  895  813 1273  869  890  852 1141  464 1240
 1357  994  721 1360 1065  408 1211 1229  626 1434 1488 1097 1443  515
  853 1142  655 1115  427  653  989 1435 1223  836 1195 1339  664  318
 1225 1328 1082  548  132  746  776  193  397  945 1214  111  573 1153
 1400  541  432  288  669  530  632 1334  638 1093 1217 1353  120  682
  489  807  827  871  665 1040 1420  240 1280  534 1456  658  142 1127
 1031 1189 1354 1467  922  394 1312  750  441  684  249  841  147  528
  594  470  957  542  802 1355 1150 1329  959 1033 1316  364  438  689
  201 1427  857  933 1181 1395  662 1436  194  967 1496 1169 1145  630
  303 1256  440 1450 1452  465

# SCALING THE DATASET

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [9]:
col_to_scale=['PercentSalaryHike','NumCompaniesWorked','MonthlyRate','MonthlyIncome','DailyRate', 'Age', 'DistanceFromHome','YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion','YearsWithCurrManager','TotalWorkingYears','HourlyRate']

In [10]:
df[col_to_scale] = scaler.fit_transform(df[col_to_scale])

In [11]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.547619,1,2,0.715820,2,0.000000,2,1,1,1,...,1,80,0,0.200,0,1,0.15,0.222222,0.000000,0.294118
1,0.738095,0,1,0.126700,1,0.250000,1,1,1,2,...,4,80,1,0.250,3,3,0.25,0.388889,0.066667,0.411765
2,0.452381,1,2,0.909807,1,0.035714,2,4,1,4,...,2,80,0,0.175,3,3,0.00,0.000000,0.000000,0.000000
3,0.357143,0,1,0.923407,1,0.071429,4,1,1,5,...,3,80,0,0.200,3,3,0.20,0.388889,0.200000,0.000000
4,0.214286,0,2,0.350036,1,0.035714,1,3,1,7,...,4,80,1,0.150,3,3,0.05,0.111111,0.133333,0.117647


In [12]:
for col in df:
    print(f'{col}: {df[col].unique()}') 

Age: [0.54761905 0.73809524 0.45238095 0.35714286 0.21428571 0.33333333
 0.97619048 0.28571429 0.47619048 0.42857143 0.4047619  0.26190476
 0.30952381 0.38095238 0.23809524 0.0952381  0.83333333 0.14285714
 0.07142857 0.57142857 0.61904762 0.66666667 0.5        0.5952381
 0.76190476 0.19047619 0.71428571 0.88095238 0.64285714 0.9047619
 0.11904762 0.78571429 0.52380952 0.85714286 0.95238095 0.04761905
 0.16666667 0.02380952 0.92857143 0.80952381 0.69047619 0.
 1.        ]
Attrition: [1 0]
BusinessTravel: [2 1 0]
DailyRate: [7.15819613e-01 1.26700072e-01 9.09806729e-01 9.23407301e-01
 3.50035791e-01 6.46385111e-01 8.74731568e-01 8.99069435e-01
 8.16034359e-02 8.56836077e-01 5.06084467e-01 3.65068003e-02
 4.06585540e-01 8.90479599e-01 7.15819613e-04 9.21259843e-01
 1.66070150e-01 7.30851825e-01 7.99570508e-01 1.92555476e-01
 4.08732999e-01 7.98854689e-01 2.26914817e-01 2.06871868e-01
 4.27344309e-01 8.44667144e-01 7.32283465e-01 4.21617752e-01
 2.68432355e-01 4.31639227e-01 5.88403722e-0

# SPLITTING THE DATASET INTO TRAINING AND TEST SET

In [13]:
x = df.drop(['Attrition'],axis = 1)
y = df['Attrition']

In [14]:
seed = 4
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=10, stratify=y )

In [15]:
df.describe()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,...,1470.000000,1470.0,1470.000000,1470.00000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,0.450567,0.161224,1.607483,0.501421,1.260544,0.292590,2.912925,2.247619,1.0,1024.865306,...,2.712245,80.0,0.793878,0.28199,2.799320,2.761224,0.175204,0.234958,0.145850,0.242537
std,0.217509,0.367863,0.665455,0.288840,0.527792,0.289531,1.024165,1.331369,0.0,602.024335,...,1.081209,0.0,0.852077,0.19452,1.289271,0.706476,0.153163,0.201285,0.214829,0.209890
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.000000,...,1.000000,80.0,0.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.285714,0.000000,1.000000,0.259843,1.000000,0.035714,2.000000,1.000000,1.0,491.250000,...,2.000000,80.0,0.000000,0.15000,2.000000,2.000000,0.075000,0.111111,0.000000,0.117647
50%,0.428571,0.000000,2.000000,0.501074,1.000000,0.214286,3.000000,2.000000,1.0,1020.500000,...,3.000000,80.0,1.000000,0.25000,3.000000,3.000000,0.125000,0.166667,0.066667,0.176471
75%,0.595238,0.000000,2.000000,0.755190,2.000000,0.464286,4.000000,3.000000,1.0,1555.750000,...,4.000000,80.0,1.000000,0.37500,3.000000,3.000000,0.225000,0.388889,0.200000,0.411765
max,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,5.000000,5.000000,1.0,2068.000000,...,4.000000,80.0,3.000000,1.00000,6.000000,4.000000,1.000000,1.000000,1.000000,1.000000


# Bagging Classifier

In [26]:
# initialize the base classifier
base_cls = DecisionTreeClassifier()
  
# no. of base classifier
num_trees = 50
  
# bagging classifier
model = BaggingClassifier(base_estimator = base_cls,n_estimators = num_trees, max_samples =.5 , max_features = 1.0, random_state = 2021)

model.fit(xtrain,ytrain)
# Accuracy on Training 
predictions_train = model.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = model.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


Training Accuracy :  0.9506802721088435

Testing Accuracy :  0.8571428571428571
Accuracy: 0.856 (0.016)


# Model learning ( RF )

In [21]:
from sklearn.ensemble import RandomForestClassifier
rnf = RandomForestClassifier(n_estimators=125, random_state=50)

In [50]:
rnf.fit(xtrain,ytrain)

RandomForestClassifier(n_estimators=125, random_state=50)

In [51]:
# Accuracy on Training 
predictions_train = rnf.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = rnf.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  1.0

Testing Accuracy :  0.8707482993197279


# SVM

In [53]:
svm_model = SVC(gamma = 'auto')
svm_model.fit(xtrain,ytrain)

SVC(gamma='auto')

In [54]:
# Accuracy on Training 
predictions_train = svm_model.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = svm_model.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  0.891156462585034

Testing Accuracy :  0.8367346938775511


# Logistic Regression

In [59]:
from sklearn.linear_model import LogisticRegression
lrmodel = LogisticRegression()
lrmodel.fit(xtrain,ytrain)

F:\New_Anaconda_Inst_Loc\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [60]:
# Accuracy on Training 
predictions_train = lrmodel.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = lrmodel.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  0.8698979591836735

Testing Accuracy :  0.8605442176870748


# Use of Ensemble with undersampling

In [67]:
df.Attrition.value_counts()

0    1233
1     237
Name: Attrition, dtype: int64

In [68]:
1233/5

246.6

In [70]:
ytrain.value_counts()

0    986
1    190
Name: Attrition, dtype: int64

In [72]:
df1 = xtrain.copy()
df1['Attrition'] = ytrain

In [74]:
df1.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
1124,0.500000,2,0.973515,2,0.178571,3,3,1,1588,4,...,80,1,0.275,3,3,0.175,0.333333,0.466667,0.352941,0
462,0.380952,2,0.111668,2,0.714286,4,1,1,621,4,...,80,0,0.250,3,3,0.250,0.388889,0.333333,0.411765,0
1424,0.404762,2,0.925555,1,0.285714,4,3,1,2008,2,...,80,0,0.250,5,3,0.250,0.388889,0.000000,0.470588,0
801,0.761905,1,0.613457,2,0.000000,4,4,1,1113,4,...,80,0,0.125,4,3,0.000,0.000000,0.000000,0.000000,1
274,0.071429,2,0.639943,1,0.071429,2,3,1,379,4,...,80,0,0.075,4,4,0.075,0.111111,0.066667,0.000000,0


In [ ]:
model1 --> class1(237) + class0(0, 237)

model2 --> class1(237) + class0(238, 484)

model3 --> class1(237) + class0(485, 730)

model3 --> class1(237) + class0(731, 976)

model3 --> class1(237) + class0(977, 1233)

In [77]:
df1_class0 = df1[df1.Attrition==0]
df1_class1 = df1[df1.Attrition==1]

In [78]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    xtrain = df_train.drop('Attrition', axis='columns')
    ytrain = df_train.Attrition
    return xtrain, ytrain    

In [79]:
xtrain, ytrain = get_train_batch(df1_class0, df1_class1, 0, 237)
rnf.fit(xtrain,ytrain)
# Accuracy on Training 
predictions_train = rnf.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = rnf.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  1.0

Testing Accuracy :  0.7278911564625851


In [81]:
xtrain, ytrain = get_train_batch(df1_class0, df1_class1, 238, 484)
lrmodel.fit(xtrain,ytrain)
# Accuracy on Training 
predictions_train = lrmodel.predict(xtrain)
print("\nTraining Accuracy : ", accuracy_score(ytrain,predictions_train))
# Accuracy on Testing
predictions_test = lrmodel.predict(xtest)
print("\nTesting Accuracy : ", accuracy_score(ytest,predictions_test))


Training Accuracy :  0.7614678899082569

Testing Accuracy :  0.7414965986394558


F:\New_Anaconda_Inst_Loc\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
